In [67]:
import pandas as pd
import os
import numpy as np

# 读取train.csv为pandas.DataFrame
train = pd.read_csv(os.getcwd() + '\\data\\train.csv')

# 数据预处理
train['Age'] = train['Age'].fillna(train['Age'].mean())
train['Cabin'] = pd.factorize(train.Cabin)[0]
train.fillna(0, inplace=True)
train['Sex'] = [1 if i=='male' else 0 for i in train.Sex]

# 处理Pclass
train['P1'] = np.array(train['Pclass']==1).astype(np.int32)
train['P2'] = np.array(train['Pclass']==2).astype(np.int32)
train['P3'] = np.array(train['Pclass']==3).astype(np.int32)
del train['Pclass']

# 处理Embarked
train.Embarked.unique()

train['E1'] = np.array(train['Embarked']=='S').astype(np.int32)
train['E2'] = np.array(train['Embarked']=='C').astype(np.int32)
train['E3'] = np.array(train['Embarked']=='Q').astype(np.int32)
del train['Embarked']

train.columns

train_x = train[['Sex', 'Age', 'SibSp', 'Parch',
       'Fare', 'Cabin', 'P1', 'P2', 'P3', 'E1', 'E2', 'E3']]
train_y_ = train['Survived'].values.reshape(len(train), 1)

import tensorflow as tf

x = tf.placeholder(tf.float32, shape=[None, 12])
y_ = tf.placeholder(tf.float32, shape=[None, 1])

w = tf.Variable(tf.random_normal([12, 1]))
b = tf.Variable(tf.random_normal([1]))
y = tf.matmul(x, w) + b
pred = tf.cast(tf.sigmoid(y) > 0.5, tf.float32)

# 定义损失函数loss
loss = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(labels=y_, logits=y))
# 定义反向传播方法
train_step = tf.train.GradientDescentOptimizer(0.0003).minimize(loss)
# 定义准确率
accuracy = tf.reduce_mean(tf.cast(tf.equal(pred, y_), tf.float32))
STEPS = 25000
BATCH_SIZE = 100

In [65]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    train_loss = []
    train_acc = []
    index = np.random.permutation(len(train_x))
    a = np.array([1, 2])
    print(train.take(index))

Int64Index([501,  67, 273, 829, 441, 502,  86, 495,  57, 258,
            ...
            450, 327, 473, 185, 830, 620,  16, 546, 122, 505],
           dtype='int64', length=891)


In [68]:

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    train_loss = []
    train_acc = []
    for i in range(1001):
        index = np.random.permutation(len(train_y_))
        X = train_x.take(index)
        Y_ = train_y_.take(index)
        for j in range(len(train_y_) // 100 + 1):
            start = j * BATCH_SIZE
            end = start + BATCH_SIZE
            sess.run(
                train_step,
                feed_dict={
                    x: X[start:end],
                    y_: Y_[start:end]
                })
            if i % 1000 == 0:
                train_loss_temp = sess.run(
                    loss,
                    feed_dict={
                        x: X[start:end],
                        y_: Y_[start:end]
                    })
                train_loss.append(train_loss_temp)
                train_acc_temp = sess.run(
                    accuracy,
                    feed_dict={
                        x: X[start:end],
                        y_: Y_[start:end]
                    })
                train_acc.append(train_acc_temp)
                print(train_loss_temp, ' ', train_acc_temp)

ValueError: Cannot feed value of shape (100,) for Tensor 'Placeholder_11:0', which has shape '(?, 1)'